<a href="https://colab.research.google.com/github/viniciusrdo/workshop-engdados/blob/main/WorkshopEngDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [2]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [5]:
# Configurando as variáveis de ambiente para o Java e Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [10]:
# Instalando biblioteca do buscador e buscando a instalação do Spark
!pip install -q findspark
import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.2-bin-hadoop2.7'

In [11]:
# Importando e criando a SparkSession
# a SparkSession é o nosso ponto de entrada para as APIs do framework Spark

from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Printando as informações da session criada
spark      